<a href="http://laf-fabric.readthedocs.org/en/latest/" target="_blank"><img align="left" src="images/laf-fabric-xsmall.png"/></a>
<a href="http://emdros.org" target="_blank"><img align="left" src="files/images/Emdros-xsmall.png"/></a>
<a href="http://www.persistent-identifier.nl/?identifier=urn%3Anbn%3Anl%3Aui%3A13-048i-71" target="_blank"><img align="left"src="images/etcbc4easy-small.png"/></a>
<a href="http://www.godgeleerdheid.vu.nl/etcbc" target="_blank"><img align="right" src="images/VU-ETCBC-xsmall.png"/></a>
<a href="http://tla.mpi.nl" target="_blank"><img align="right" src="images/TLA-xsmall.png"/></a>
<a href="http://www.dans.knaw.nl" target="_blank"><img align="right"src="images/DANS-xsmall.png"/></a>

# Versions

In [1]:
import sys
import collections
db = 'shebanq_web'
sversion = 'etcbc4'
datadir = '/Users/dirk/SURFdrive/laf-fabric-output/{}/shebanq'.format(sversion)
shebanqdir = '/Users/dirk/SURFdrive/current/demos/github/shebanq/modules'

This notebooks defines a new datamodel for queries in SHEBANQ, and it converts data according to the old model to data according to the new model.

The old model had no provisions for working with queries on multiple versions of the data. The new model remedies that.

# The old model

    create table queries(
        id int(4) primary key,
        name varchar(512),
        description longtext,
        mql longtext,
        is_active char(1),
        created_on datetime,
        created_by int(4),
        modified_on datetime,
        modified_by int(4),
        project int(4),
        organization int(4),
        is_published char(1),
        executed_on date_time,
        foreign key (created_by) references auth_user(id),
        foreign key (modified_by) references auth_user(id),
        foreign key (organization) references organization(id),
        foreign key (project) references project(id)
    );

# To be changed

We will call the table ``query`` instead of ``queries``.
The fields ``is_active`` and ``modified_by`` are not used. 
We will strip them from the table.

More importantly, we split the query record into a part that is generic to all versions of the data, and parts that
are dependent on the particular version of the data.

We rename the field ``is_published`` to ``is_shared``. You can share a query, with all its execution records.
Other people can then see what you are doing, in all versions of the data.
When you publish a query with its execution on a single version of the data you freeze the results and the query body.

We add an attribute ``is_published`` to the part that is version dependent, to indicate that the execution results of this query on that version are to be frozen: no change allowed, not even by the creator.
A query that has one of its versions ``is_published`` must remain ``is_shared``.


# The new model



In [2]:
create_text = dict(
    query='''
create table query(
    id int(4) primary key auto_increment,
    name varchar(512),
    description longtext,
    created_on datetime,
    created_by int(4),
    modified_on datetime,
    shared_on datetime,
    is_shared char(1),
    project int(4),
    organization int(4),
    foreign key (created_by) references auth_user(id),
    foreign key (organization) references organization(id),
    foreign key (project) references project(id)
) character set utf8;
''',
    query_exe='''
create table query_exe(
    id int(4) primary key auto_increment,
    mql longtext,
    version varchar(32),
    resultmonads int(4),
    results int(4),
    executed_on datetime,
    modified_on datetime,
    is_published char(1),
    published_on datetime,
    query_id int(4),
    foreign key (query_id) references query(id)
) character set utf8;
''',
    monads='''
create table monads(
    query_exe_id int(4),
    first_m int(4),
    last_m int(4),
    foreign key (query_exe_id) references query_exe(id)
) character set utf8;
''',
)

# Notes on the new model

1. The name ``query_exe`` indicates that this table consists of query *execution* records. An execution of a query is done on a concrete version of the data, at a definite time.
1. There are now two fields modified_on. The one in ``query`` essentially indicates the time of change of the description or name. The time of change of project and organization are not recorded. The one in ``query_exe`` denotes the time that the query body was last modified.
1. There is a new field ``published_on`` to record the time that a query is published.
1. There is a new field ``resultmonads``: it stores the total number of monads (=words) in all results of the query
1. There is a new field ``results``: it stores the number of logical results of the query.
1. The field ``mql`` (the query body) corresponds to the *version* of the data. It is conceivable that the same intention must be expressed differently for different versions of the data.
1. The records in table ``monadsets`` now reference a ``query_exe`` record instead of a query record.

# Conversion

The conversion from old to new data goes as follows:

1. Dump the old queries table from the shebanq database
1. Dump the old monadsets table from the shebanq database
1. Transform the queries data into query and query_exe data
1. For queries that were *public* in the old model:
   1. They become *shared* in the new model, and none of the new versions will be initially *published* 
      in the new model
   1. If the query results are up to date we do the following:
      1. we create a version record for version 4b, and copy the MQL body also to that version, with a modified date.          We do not execute it
      1. we execute the query in version 4 and determine the results and resultmonads and fill that in
1. Compile a mapping from ``query`` ids to corresponding ``query_exe`` ids
1. Transform the monadsets table into a new one, where the field ``query_id`` has been replaced with the correct ``query_exe`` id, i.e. according to the mapping just made
1. Write the new data out into an sql file
1. Import the new sql file into the shebanq_web database


# Table reading

Dump the old queries table from mysql using the following commands

In [3]:
%cd {datadir}
!chmod a+rwx .
!mysqldump -u root --tab . {db} queries
!mysqldump -u root --tab . {db} monadsets
!ls -l queries* monadsets*

/Users/dirk/SURFdrive/laf-fabric-output/etcbc4/shebanq
-rw-r--r--  1 dirk    staff     1610 Apr  8 17:42 monadsets.sql
-rw-rw-rw-  1 _mysql  staff  8832363 Apr  8 17:42 monadsets.txt
-rw-r--r--  1 dirk    staff     2362 Apr  8 17:42 queries.sql
-rw-rw-rw-  1 _mysql  staff   314672 Apr  8 17:42 queries.txt
-rw-rw-rw-  1 dirk    wheel   275177 Mar 19 07:47 queries_old.txt


In [3]:
old_fields = (
    'id',
    'name',
    'description',
    'mql',
    'is_active',
    'created_on',
    'created_by',
    'modified_on',
    'modified_by',
    'project',
    'organization',
    'is_published',
    'executed_on',
)
new_fields = dict(
    query=(
        'id',
        'name',
        'description',
        'created_on',
        'created_by',
        'modified_on',
        'shared_on',
        'is_shared',
        'project',
        'organization',
    ),
    query_exe=(
        'id',
        'mql',
        'version',
        'resultmonads',
        'results',
        'executed_on',
        'modified_on',
        'is_published',
        'published_on',
        'query_id',
    ),
    monads=(
        'query_exe_id',
        'first_m',
        'last_m',
    ),
)
                  
nold_fields = len(old_fields)
oldf = open('{}/queries.txt'.format(datadir), newline='\n')
olddata = oldf.read().rstrip('\n')
olddata_le = olddata.replace('\r', '').replace('\\\n', '▼').replace('\\\t','▶').split('\n')
lineno = 0
errors = []
old_records = []
for line in olddata_le:
    lineno += 1
    values = line.split('\t')
    if len(values) != nold_fields:
        errors.append((lineno, '{} fields encountered instead of {}'.format(len(values), nold_fields)))
        continue
    escaped_values = [v.replace('▶', '\t').replace('▼', '\n') for v in values]
    old_records.append(escaped_values)
print('{:>3} lines'.format(lineno))
print('{:>3} errors'.format(len(errors)))
for (l, e) in errors:
    print('{:>3}: {}'.format(l,e))
print("Done")
oldf.close()

534 lines
  0 errors
Done


# Table filling

We compose all the records for all the tables.

In [4]:
print(old_records[0])

['10', 'lexeme search', 'SELECT ALL OBJECTS \nWHERE \n[Word Focus sp = verb AND lex = "KTB["]', "SELECT ALL OBJECTS \nWHERE \n[Word Focus sp = verb AND lex ~ '^KTB\\\\[.*$' GET lex]", 'T', '2014-07-22 21:31:58', '4', '2014-07-22 22:08:41', '4', '1', '2', 'T', '\\N']


## Pass 1: Generate the new identifiers

In our first pass we compute what the identifiers for our new records are going to be, in order to print a list of query_exe identifiers and MQL bodies.

We then can run those queries in order to compute the number of results.

With the outcome we can undertake the full generation of the records in pass 2.

In [5]:
print("Pass 1: Generate the identifiers ... ")
ofi = dict((n, i) for (i, n) in enumerate(old_fields))

def s_esc(sql, as_str=False):
    val = str(sql)
    if val == '\\N':
        return 'null'
    val = val.replace("'", "''").replace('\\','\\\\').replace('\n','\\n').replace('\t', '\\t')
    if as_str:
        return "'{}'".format(val)
    return val


nq = 0
qe = 0
qr = 0
qmapping = {}
needtorun = collections.defaultdict(lambda: [0, set()])

for r in old_records:
    nq += 1
    qid = r[ofi['id']]
    qe += 1
    qmapping[qid] = str(qe)
    if r[ofi['is_published']] == 'T' and r[ofi['executed_on']] and r[ofi['executed_on']] >= r[ofi['modified_on']]:
        qr += 1
        mql = r[ofi['mql']]
        name = r[ofi['name']]
        needtorun[mql][0] = '{}..........{}'.format(qid, name)
        needtorun[mql][1].add(qe)
        qe += 1

mqf = open('{}/mql.txt'.format(datadir), 'w')
for mql in needtorun:
    qinfo = needtorun[mql]
    mqf.write('{}\n----------\n{}\n----------\n{}\n==========\n'.format(mql, qinfo[0], ','.join(str(x) for x in qinfo[1])))
mqf.close()
print('{} query records and {} query_exe records\nNeed to run {} queries for {} query_exe records'.format(
        nq, qe, len(needtorun), qr,
))

Pass 1: Generate the identifiers ... 
534 query records and 819 query_exe records
Need to run 267 queries for 285 query_exe records


# Running old queries

We need to run old published queries with up-to-date results in order to collect the number of results and resultmonads.

The Python bindings for Emdros are available for Python2 only.

In [6]:
!python2 {shebanqdir}/convert.py {datadir}

Reading data from /Users/dirk/SURFdrive/laf-fabric-output/etcbc4/shebanq/mql.txt
  0.00s   1 qid=531 Double imperatives             ...  1.82s OK - 56 - 107
  1.82s   2 qid=596 Grammatical Terms: object cla  ...  0.42s OK - 5 - 49
  2.23s   3 qid= 26 Ayin"Waw/Jod verb              ...  0.80s OK - 19 - 19
  3.04s   4 qid=525 HNH+J with perfect in Isa 28:  ...  3.24s OK - 4 - 8
  6.28s   5 qid=578 Wayyiqtol in Genesis 27        ...  0.01s OK - 73 - 73
  6.29s   6 qid=597 Grammatical Terms: construct   ...  1.11s OK - 84 - 192
  7.40s   7 qid=593 Grammatical Terms: nominal cl  ...  0.31s OK - 31 - 138
  7.71s   8 qid= 98 QR> Psalms                     ...  0.23s OK - 19 - 17
  7.94s   9 qid=162 Seungho Park                   ...  0.55s OK - 819 - 819
  8.49s  10 qid=569 Motion verbs with toponym as   ...  2.25s OK - 162 - 316
    11s  11 qid=506 OTST619 <Su> of to bless (pie  ...  3.13s OK - 72 - 166
    14s  12 qid=492 tutorial: clause connections   ...  7.40s OK - 5 - 21
    21s  13 qid

# Gather the execution results

In [7]:
mqf = open('{}/results.txt'.format(datadir))
qrindex = {}
for line in mqf:
    (ix, r, rm) = line.rstrip('\n').split('\t')
    qrindex[int(ix)] = (r, rm)
mqf.close()

In [8]:
print("Pass 2: Fill the tables ... ")

tables = dict(query=[], query_exe=[], monads=[])
tb_q = tables['query']
tb_qx = tables['query_exe']

qe = 0

for r in old_records:
    qid = r[ofi['id']]
    new_query = (
        s_esc(qid),
        s_esc(r[ofi['name']], True),
        s_esc(r[ofi['description']], True),
        s_esc(r[ofi['created_on']], True),
        s_esc(r[ofi['created_by']]),
        s_esc(r[ofi['modified_on']], True),
        s_esc(r[ofi['modified_on']], True),
        s_esc(r[ofi['is_published']], True),
        s_esc(r[ofi['project']]),
        s_esc(r[ofi['organization']]),
    )
    tb_q.append('({})'.format(','.join(new_query)))
    qe += 1
    qmapping[qid] = str(qe)
    if qe in qrindex:
        (rx, rm) = qrindex[qe]
    else:
        (rx, rm) = ('\\N', '\\N')
    new_query_exe = (
        s_esc(qe),
        s_esc(r[ofi['mql']], True),
        s_esc('4', True),
        s_esc(rm),
        s_esc(rx),
        s_esc(r[ofi['executed_on']], True),
        s_esc(r[ofi['modified_on']], True),
        s_esc('\\N'),
        s_esc('\\N'),
        s_esc(r[ofi['id']]),
    )
    tb_qx.append('({})'.format(','.join(new_query_exe)))
    if r[ofi['is_published']] == 'T' and r[ofi['executed_on']] and r[ofi['executed_on']] >= r[ofi['modified_on']]:
        qe += 1
        new_query_exe = (
            s_esc(qe),
            s_esc(r[ofi['mql']], True),
            s_esc('4b', True),
            s_esc('\\N'),
            s_esc('\\N'),
            s_esc('\\N'),
            s_esc(r[ofi['modified_on']], True),
            s_esc('\\N'),
            s_esc('\\N'),
            s_esc(r[ofi['id']]),
        )
        tb_qx.append('({})'.format(','.join(new_query_exe)))
print('{} query records and {} query_exe records'.format(len(tb_q), len(tb_qx)))

Pass 2: Fill the tables ... 
534 query records and 819 query_exe records


# Transforming the monadsets

Here we read the old ``monadsets`` table, and replace the contents of the ``query_id`` field, which is the id of an associated query, by the id of the new ``query_exe`` record that belongs to that query.
We find the new id by means of the ``qmapping`` dictionary that we have just constructed.

In [9]:
oldf = open('{}/monadsets.txt'.format(datadir), newline='\n')
tb_m = tables['monads']
for line in oldf:
    (mid, qid, fm, lm) = line.rstrip('\n').split('\t')
    tb_m.append('({})'.format(','.join((qmapping[qid], str(fm), str(lm)))))
print('{} monads records'.format(len(tb_m)))

351657 monads records


# SQL generation

In [10]:
limits_row = dict(
    query=100,
    query_exe=100,
    monads=10000,
)
table_order = ('query', 'query_exe', 'monads')
sqf = open('{}/queryx.sql'.format(datadir), 'w')
sqf.write('''
use {}
'''.format(db)
)
for table in reversed(table_order):
    sqf.write('''
drop table if exists `{}`;
'''.format(table))

for table in table_order:
    sqf.write(create_text[table])

for table in table_order:
    limit_row = limits_row[table]
    w = 0
    start = '''insert into {} ({}) values
'''.format(table, ','.join(new_fields[table]))
    rows = tables[table]
    r = 0
    while r < len(rows):
        sqf.write(start)
        s = min(r + limit_row, len(rows))
        sqf.write(' {}'.format(rows[r]))
        w += 1
        if r + 1 < len(rows):
            for t in rows[r + 1:s]: 
                sqf.write('\n,{}'.format(t))
                w += 1
        sqf.write(';\n')
        r = s
    print('Table {}: written {} rows'.format(table, w))
sqf.close()
print("Done")

Table query: written 534 rows
Table query_exe: written 819 rows
Table monads: written 351657 rows
Done


# Table importing

In [12]:
%cd {datadir}
!mysql -u root <queryx.sql

/Users/dirk/SURFdrive/laf-fabric-output/etcbc4/shebanq


Check the new monads table by running this query, which should have no results:

    select
        monads.first_m - monadsets.first_m,
        monads.last_m - monadsets.last_m
    from monads inner join monadsets on 
        monads.id = monadsets.id
    where 
        monads.first_m - monadsets.first_m != 0 or 
        monads.last_m - monadsets.last_m != 0
    ;

# Final actions

If all went well, drop the old tables: ``queries`` and ``monadsets``.
But make a backup of the original database, because otherwise you cannot redo this conversion.

# 2015-05-23
We add the version of the query engine (emdros) to the metadata of the query.
First we alter the table query_exe, to contain a column eversion varchar(32).
Then we add the initial value "3.4.0" to all records that have the field executed_on filled in.